## Import necessary packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"
from kilt import retrieval
from kilt import kilt_utils as utils
import tasks
# from kilt.retrievers import DPR_connector
import utils
from rouge_score import rouge_scorer
import random
import numpy as np
import torch
torch.set_grad_enabled(False)
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from datasets import load_dataset
import json
from tqdm import tqdm
import opensource

np.random.seed(42)

/home/lishuo1/anaconda3/envs/kilt37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

def save_results(task):
    # save retrieved_scores to a pickle file
    write_list(retrieved_scores, f'retrieved_scores_{task}.p')
    # save retrieved_true_scores to a pickle file
    write_list(retrieved_true_scores, f'retrieved_true_scores_{task}.p')
    # save queries to a pickle file
    write_list(queries, f'queries_{task}.p')
    # save answers to a pickle file
    write_list(answers, f'answers_{task}.p')
    # save passages to a pickle file
    write_list(passages, f'passages_{task}.p')
    # save opensource_true_scores to a pickle file
    write_list(opensource_true_scores, f'opensource_true_scores_{task}.p')
    # save opensource_texts to a pickle file
#     write_list(opensource_texts, f'opensource_texts_{task}.p')
    # save opensource_answers to a pickle file
    write_list(opensource_answers, f'opensource_answers_{task}.p')
    # save opensource_semantics to a picle file
    write_list(opensource_semantics, f'opensource_semantics_{task}.p')
    # save feasibilities to a pickle file
    write_list(feasibilities, f'feasibilities_{task}.p')
    # save occurances to a pickle file
    write_list(occurances, f'occurances_{task}.p')
    # save semantic_ids to a pickle file
    write_list(semantic_ids, f'semantic_ids_{task}.p')

def read_results(task):
    retrieved_scores = read_list(f'retrieved_scores_{task}.p')
    retrieved_true_scores = read_list(f'retrieved_true_scores_{task}.p')
    queries = read_list(f'queries_{task}.p')
    answers = read_list(f'answers_{task}.p')
    opensource_true_scores = read_list(f'opensource_true_scores_{task}.p')
    opensource_answers = read_list(f'opensource_answers_{task}.p')
    opensource_semantics = read_list(f'opensource_semantics_{task}.p')
    opensource_occurances = read_list(f'occurances_{task}.p')
    opensource_semantic_ids = read_list(f'semantic_ids_{task}.p')
    
    return retrieved_scores, retrieved_true_scores, \
           queries, answers, \
           opensource_true_scores, opensource_answers, \
           opensource_occurances, opensource_semantic_ids

In [71]:
task = 'trivia'

In [72]:
semantic = False
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)
if semantic:
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    # setup semantic model
    semantic_tokenizer = \
        AutoTokenizer.from_pretrained("microsoft/deberta-large-mnli")
    semantic_model = \
        AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-large-mnli"
        ).cuda()

In [73]:
retrieved_scores, retrieved_true_scores, queries, answers, opensource_true_scores, opensource_answers, opensource_occurances, opensource_semantic_ids = \
read_results(task)

In [74]:
len(retrieved_true_scores)

518

In [62]:
indices = np.arange(len(retrieved_true_scores)-1)
random.shuffle(indices)
cal_first_indices = indices[:int(len(indices) * 0.3)]
cal_second_indices = indices[int(len(indices) * 0.3) : int(len(indices) * 0.6)]
test_indices = indices[int(len(indices) * 0.6):]

cal_first_retrieved_true_scores = utils.split(retrieved_true_scores, cal_first_indices)
cal_second_retrieved_true_scores = utils.split(retrieved_true_scores, cal_second_indices)
test_retrieved_true_scores = utils.split(retrieved_true_scores, test_indices)
cal_first_opensource_true_scores = utils.split(opensource_true_scores, cal_first_indices)
cal_second_opensource_true_scores = utils.split(opensource_true_scores, cal_second_indices)
test_opensource_true_scores = utils.split(opensource_true_scores, test_indices)
cal_first_retrieved_scores = utils.split(retrieved_scores, cal_first_indices)
cal_second_retrieved_scores = utils.split(retrieved_scores, cal_second_indices)
test_retrieved_scores = utils.split(retrieved_scores, test_indices)
cal_first_opensource_occurances = utils.split(opensource_occurances, cal_first_indices)
cal_second_opensource_occurances = utils.split(opensource_occurances, cal_second_indices)
test_opensource_occurances = utils.split(opensource_occurances, test_indices)
cal_first_opensource_semantic_ids = utils.split(opensource_semantic_ids, cal_first_indices)
cal_second_opensource_semantic_ids = utils.split(opensource_semantic_ids, cal_second_indices)
test_opensource_semantic_ids = utils.split(opensource_semantic_ids, test_indices)
cal_first_queries = utils.split(queries, cal_first_indices)
cal_second_queries = utils.split(queries, cal_second_indices)
test_queries = utils.split(queries, test_indices)
cal_first_opensource_answers = utils.split(opensource_answers, cal_first_indices)
cal_second_opensource_answers = utils.split(opensource_answers, cal_second_indices)
test_opensource_answers = utils.split(opensource_answers, test_indices)


In [33]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)

## Compute coverage rate

In [9]:
def coverage(
        retrieved_true_scores_list, opensource_true_scores_list,
        retrieved_thr, qa_thr):

    includes = []
    for idx, (retrieved_true_score, opensource_true_score) in enumerate(zip(retrieved_true_scores_list, opensource_true_scores_list)):
#         if idx > 20:
        opensource_true_score = np.max(opensource_true_score)
        include = True if (retrieved_true_score >= retrieved_thr and 
                           opensource_true_score >= qa_thr) \
                       else False
        includes.append(include)
    return includes

In [65]:
alpha = 0.1
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha/2)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha/2)

retrieved_coverage = np.mean(np.array(cal_second_retrieved_true_scores) >= retrieved_thr)
cal_second_scores = []
for scores in cal_second_opensource_true_scores:
    cal_second_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(cal_second_scores) >= opensource_qa_thr)
print('retrieval coverage', retrieved_coverage)
print('qa coverage', qa_coverage)

retrieved_coverage = np.mean(np.array(test_retrieved_true_scores) >= retrieved_thr)
test_scores = []
for scores in test_opensource_true_scores:
    test_scores.append(np.max(scores))
qa_coverage = np.mean(np.array(test_scores) >= opensource_qa_thr)
print('test retrieval coverage', retrieved_coverage)
print('test qa coverage', qa_coverage)

retrieval coverage 0.9444444444444444
qa coverage 1.0
test retrieval coverage 0.9637305699481865
test qa coverage 1.0


In [66]:
coverages = coverage(test_retrieved_true_scores, 
                    test_opensource_true_scores,
                    retrieved_thr,
                    opensource_qa_thr
                   )
print('End-to-end coverage', np.mean(coverages))

End-to-end coverage 0.9637305699481865


## Bayesian optimization

In [13]:
from skopt.space import Real
from skopt import gp_minimize
from skopt.utils import use_named_args

In [14]:
def evaluate(
        test_retrieved_scores,
        test_queries, test_opensource_answers, 
        test_opensource_occurances, test_opensource_semantic_ids,
        retrieved_thr, opensource_qa_thr):

    cluster = False
    includes = []
    answer_counts = []
    semantic_counts = []
    includes = []
    coverages = []
    for idx, (retrieved_scores_tmp, \
            query_tmp, opensource_answers_tmp, \
            opensource_true_scores_tmp, \
            opensource_occurances_tmp, opensource_semantic_ids_tmp) \
        in enumerate(zip(
            test_retrieved_scores, \
            test_queries, test_opensource_answers, \
            test_opensource_true_scores, \
            test_opensource_occurances, test_opensource_semantic_ids)):
        include = False
        retrieved_count = 0
        answers_tmp = []
        semantics = []
        for retrieved_score, answer, item_occurance, semantic_set_ids in zip(retrieved_scores_tmp, opensource_answers_tmp, opensource_occurances_tmp, opensource_semantic_ids_tmp):
            if retrieved_score < retrieved_thr:
                continue
            else:
                retrieved_count += 1
                for predicted_answer in semantic_set_ids.keys():
                    concept_id = semantic_set_ids[predicted_answer]
                    repeat = item_occurance[predicted_answer]
                    prob = repeat / len(answer)
                    if prob >= opensource_qa_thr:
                        answers_tmp.extend([predicted_answer] * repeat)
                        semantics.append(predicted_answer)
                        
                        # TODO: check if the semantic is consistent with true answer
                        include = True
        if cluster:
            semantic_set_ids, semantic_probs, item_occurance = \
                        utils.clustering(semantics, "", scorer=scorer)
                    
            answer_counts.append(len(answers_tmp))
            semantic_counts.append(len(semantic_probs.keys()))
        else:
            answer_counts.append(len(answers_tmp))
            semantic_counts.append(len(semantics))
        includes.append(include)
        
    return [includes, answer_counts, semantic_counts]

In [15]:
results = evaluate(
    test_retrieved_scores, test_queries,
    test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    retrieved_thr, opensource_qa_thr)

In [40]:
"""
Weight HMP module
"""
w1 = Real(name='w1', low=0.0, high=1.0)
w2 = Real(name='w2', low=0.0, high=1.0)

def softmax(vec):
    nom = np.exp(vec - np.mean(vec))
    return nom / np.sum(nom)

# Gather the search-space dimensions in a list.
dimensions = [w1, w2]

@use_named_args(dimensions=dimensions)
def objective(w1, w2):
    weights = softmax(np.array([w1, w2])).reshape(-1, 1)
    alpha_retrieve = alpha * weights[0]
    alpha_qa = alpha * weights[1]
    retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
    cal_first_scores = []
    for scores in cal_first_opensource_true_scores:
        cal_first_scores.append(np.max(scores))
    opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
    results = evaluate(cal_second_retrieved_scores, cal_second_queries, 
                       cal_second_opensource_answers, \
                       cal_second_opensource_occurances, cal_second_opensource_semantic_ids,
                       retrieved_thr, opensource_qa_thr)
    coverage = np.mean(results[0])
    average_answer = np.mean(results[1])
    average_semantic = np.mean(results[2])
    return average_semantic

result = gp_minimize(func=objective,
                     dimensions=dimensions,
                     acq_func="EI",      # the acquisition function
                     n_calls=10,
                     random_state=42,
                     verbose=True)

print("Best fitness:", result.fun)
print("Best parameters:", softmax(result.x))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1156
Function value obtained: 267.1074
Current minimum: 267.1074
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0805
Function value obtained: 45.6711
Current minimum: 45.6711
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0584
Function value obtained: 45.4564
Current minimum: 45.4564
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0596
Function value obtained: 46.1544
Current minimum: 45.4564
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0625
Function value obtained: 48.1678
Current minimum: 45.4564
Iteration No: 6 started

In [41]:
weights = softmax(result.x).reshape(-1, 1)
alpha_retrieve = alpha * weights[0]
alpha_qa = alpha * weights[1]
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    retrieved_thr, opensource_qa_thr)

In [42]:
print('Coverage', np.mean(results[0]))
print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

Coverage 0.9798994974874372
Average answer 204.70854271356785
Average semantic 46.93969849246231


In [43]:
weights = softmax(result.x).reshape(-1, 1)
alpha_retrieve = alpha * (1/2.0)
alpha_qa = alpha * (1/2.0)
retrieved_thr = utils.compute_threshold(cal_first_retrieved_true_scores, alpha=alpha_retrieve)
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha_qa)
results = evaluate(
    test_retrieved_scores, test_queries,
    test_opensource_answers, 
    test_opensource_occurances, test_opensource_semantic_ids,
    retrieved_thr, opensource_qa_thr)

In [44]:
print('Coverage', np.mean(results[0]))
print('Average answer', np.mean(results[1]))
print('Average semantic', np.mean(results[2]))

Coverage 0.9949748743718593
Average answer 217.72361809045225
Average semantic 49.8643216080402


In [ ]:
cal_first_scores = []
for scores in cal_first_opensource_true_scores:
    cal_first_scores.append(np.max(scores))
opensource_qa_thr = utils.compute_threshold(cal_first_scores, alpha=alpha)

for idx, opensource_answers_tmp in enumerate(test_opensource_answers):
    print(len(opensource_answers_tmp))
    semantic_set_ids, semantic_probs, item_occurance = \
            utils.clustering(opensource_answers_tmp, "", scorer=scorer)
    for predicted_answer in semantic_set_ids.keys():
        concept_id = semantic_set_ids[predicted_answer]
        repeat = item_occurance[predicted_answer]
        prob = repeat / len(answer)
        if prob >= opensource_qa_thr:
            answers_tmp.extend([predicted_answer] * repeat)
            semantics.append(predicted_answer)